In [1]:
!pip install ZODB
!pip install persistent

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.8/417.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.1/299.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 4.4 MB/s eta 0:00:00


In [2]:
from persistent.list import PersistentList
import ZODB, ZODB.FileStorage
import transaction
from datetime import datetime

from persistent import Persistent

class PersistentSet(Persistent):
    def __init__(self, iterable=None):
        # Almacena internamente el set normal de Python.
        self._data = set(iterable) if iterable is not None else set()

    def add(self, item):
        self._data.add(item)
        self._p_changed = True  # Marca el objeto como modificado.

    def remove(self, item):
        self._data.remove(item)
        self._p_changed = True

    def __contains__(self, item):
        return item in self._data

    def __iter__(self):
        return iter(self._data)

    def __len__(self):
        return len(self._data)

    def __repr__(self):
        return repr(self._data)

In [3]:
# Clase Persona
class Persona(Persistent):
    def __init__(self, cedula=None, nombre=None, apellido=None):
        self.cedula = cedula
        self.nombre = nombre
        self.apellido = apellido

    def __str__(self):
        return f"Persona [cedula={self.cedula}, nombre={self.nombre}, apellido={self.apellido}]"

# Clase Cliente que extiende Persona
class Cliente(Persona):
    def __init__(self, cedula=None, nombre=None, apellido=None, direccion=None, correoElectronico=None, telefono=None):
        super().__init__(cedula, nombre, apellido)
        self.direccion = direccion
        self.correoElectronico = correoElectronico
        self.telefono = telefono
        self.consta = PersistentSet()  # Conjunto de Ingreso

    def addIngreso(self, ingreso):
        self.consta.add(ingreso)
        ingreso.clienteRegistra = self

    def removeIngreso(self, ingreso):
        if ingreso in self.consta:
            self.consta.remove(ingreso)
            ingreso.clienteRegistra = None

    def direccionCliente(self, cedula):
        if self.cedula == cedula:
            return self.direccion
        return ""

    def __str__(self):
        return (f"Cliente [{super().__str__()}, direccion={self.direccion}, "
                f"correoElectronico={self.correoElectronico}, telefono={self.telefono}]")

# Clase Trabajador que extiende Persona
class Trabajador(Persona):
    def __init__(self, cedula=None, nombre=None, apellido=None, correoElectronico=None, telefono=None, salario=0.0):
        super().__init__(cedula, nombre, apellido)
        self.correoElectronico = correoElectronico
        self.telefono = telefono
        self.salario = salario
        self.registra = PersistentSet()  # Conjunto de Transaccion

    def addTransaccion(self, transaccion):
        self.registra.add(transaccion)
        transaccion.trabajadorRegistra = self

    def removeTransaccion(self, transaccion):
        if transaccion in self.registra:
            self.registra.remove(transaccion)
            transaccion.trabajadorRegistra = None

    def aumentarSalario(self, aumento):
        self.salario += aumento
        return f"Nuevo salario: {self.salario}"

    def obtenerSalario(self, cedula):
        if self.cedula == cedula:
            return self.salario
        return 0.0

    def __str__(self):
        return (f"Trabajador [{super().__str__()}, correoElectronico={self.correoElectronico}, "
                f"telefono={self.telefono}, salario={self.salario}]")

# Clase Administrador que extiende Persona
class Administrador(Persona):
    def __init__(self, cedula=None, nombre=None, apellido=None, correoElectronico=None, telefono=None, salario=0.0):
        super().__init__(cedula, nombre, apellido)
        self.correoElectronico = correoElectronico
        self.telefono = telefono
        self.salario = salario
        self.revisa = PersistentSet()  # Conjunto de Transaccion

    def addTransaccionRevisa(self, transaccion):
        self.revisa.add(transaccion)
        transaccion.esRevisada = self

    def removeTransaccionRevisa(self, transaccion):
        if transaccion in self.revisa:
            self.revisa.remove(transaccion)
            transaccion.esRevisada = None

    def aumentarSalario(self, aumento):
        self.salario += aumento
        return f"Nuevo salario: {self.salario}"

    def obtenerSalario(self, cedula):
        if self.cedula == cedula:
            return self.salario
        return 0.0

    def obtenerReporte(self, inicio, fin):
        reporte = PersistentSet()
        for t in self.revisa:
            if inicio <= t.fechaTransaccion <= fin:
                reporte.add(t)
        return reporte

    def __str__(self):
        return (f"Administrador [{super().__str__()}, correoElectronico={self.correoElectronico}, "
                f"telefono={self.telefono}, salario={self.salario}]")

# Clase Transaccion
class Transaccion(Persistent):
    def __init__(self, idTrans=None, monto=0.0, fechaTransaccion=None):
        self.idTrans = idTrans
        self.monto = monto
        self.fechaTransaccion = fechaTransaccion
        self.esRevisada = None          # Administrador que revisa
        self.trabajadorRegistra = None   # Trabajador que registra

    @staticmethod
    def filtrarTransacciones(transacciones, inicio, fin):
        filtradas = PersistentSet()
        for t in transacciones:
            if inicio <= t.fechaTransaccion <= fin:
                filtradas.add(t)
        return filtradas

    def __str__(self):
        return (f"Transaccion [idTrans={self.idTrans}, monto={self.monto}, "
                f"fechaTransaccion={self.fechaTransaccion}]")

# Clase Ingreso que extiende Transaccion
class Ingreso(Transaccion):
    def __init__(self, idTrans=None, monto=0.0, fechaTransaccion=None, motivoIngreso=None):
        super().__init__(idTrans, monto, fechaTransaccion)
        self.motivoIngreso = motivoIngreso
        self.clienteRegistra = None  # Cliente que registra el ingreso
        self.contiene = PersistentSet()  # Conjunto de Producto

    def addProducto(self, producto):
        self.contiene.add(producto)
        producto.addIngresoContenedor(self)

    def removeProducto(self, producto):
        if producto in self.contiene:
            self.contiene.remove(producto)
            producto.removeIngresoContenedor(self)

    @staticmethod
    def filtrarIngresos(ingresos, inicio, fin):
        filtrados = PersistentSet()
        for i in ingresos:
            if inicio <= i.fechaTransaccion <= fin:
                filtrados.add(i)
        return filtrados

    @staticmethod
    def totalIngresos(ingresos, inicio, fin):
        total = 0.0
        for i in ingresos:
            if inicio <= i.fechaTransaccion <= fin:
                total += i.monto
        return total

    def __str__(self):
        return f"Ingreso [{super().__str__()}, motivoIngreso={self.motivoIngreso}]"

# Clase Egreso que extiende Transaccion
class Egreso(Transaccion):
    def __init__(self, idTrans=None, monto=0.0, fechaTransaccion=None, motivoEgreso=None):
        super().__init__(idTrans, monto, fechaTransaccion)
        self.motivoEgreso = motivoEgreso

    @staticmethod
    def filtrarEgresos(egresos, inicio, fin):
        filtrados = PersistentSet()
        for e in egresos:
            if inicio <= e.fechaTransaccion <= fin:
                filtrados.add(e)
        return filtrados

    @staticmethod
    def totalEgresos(egresos, inicio, fin):
        total = 0.0
        for e in egresos:
            if inicio <= e.fechaTransaccion <= fin:
                total += e.monto
        return total

    def __str__(self):
        return f"Egreso [{super().__str__()}, motivoEgreso={self.motivoEgreso}]"

# Clase Producto
class Producto(Persistent):
    def __init__(self, nombreProducto=None, precio=0.0):
        self.nombreProducto = nombreProducto
        self.precio = precio
        self.esContenido = PersistentSet()  # Conjunto de Ingreso

    def addIngresoContenedor(self, ingreso):
        self.esContenido.add(ingreso)

    def removeIngresoContenedor(self, ingreso):
        if ingreso in self.esContenido:
            self.esContenido.remove(ingreso)

    def filtrarEgresos(self, inicio, fin):
        filtrados = PersistentSet()
        for i in self.esContenido:
            if inicio <= i.fechaTransaccion <= fin:
                filtrados.add(i)
        return filtrados

    def totalEgresos(self, inicio, fin):
        total = 0.0
        for i in self.esContenido:
            if inicio <= i.fechaTransaccion <= fin:
                total += i.monto
        return total

    def __str__(self):
        return f"Producto [nombreProducto={self.nombreProducto}, precio={self.precio}]"


In [4]:

# Función para insertar datos de ejemplo
def insertarDatos(root):
    # Crear clientes
    cliente1 = Cliente("001", "Juan", "Perez", "Av. Siempre Viva 123",
                       "juan@mail.com", "123456789")
    cliente2 = Cliente("002", "Maria", "Lopez", "Calle Falsa 456",
                       "maria@mail.com", "987654321")
    root['clientes'].append(cliente1)
    root['clientes'].append(cliente2)

    # Crear trabajador
    trabajador1 = Trabajador("003", "Carlos", "Sanchez", "carlos@mail.com",
                             "5555555", 1200.0)
    root['trabajadores'].append(trabajador1)

    # Crear administrador
    admin1 = Administrador("004", "Ana", "Gomez", "ana@mail.com",
                           "4444444", 2000.0)
    root['administradores'].append(admin1)

    # Crear productos
    producto1 = Producto("Vino Tinto", 15.99)
    producto2 = Producto("Cerveza", 5.99)
    root['productos'].append(producto1)
    root['productos'].append(producto2)

    # Fechas para transacciones
    fecha1 = datetime.strptime("2025-02-15", "%Y-%m-%d")
    fecha2 = datetime.strptime("2025-02-20", "%Y-%m-%d")

    # Crear ingreso
    ingreso1 = Ingreso("T001", 500.0, fecha1, "Venta de producto X")
    cliente1.addIngreso(ingreso1)
    trabajador1.addTransaccion(ingreso1)
    ingreso1.addProducto(producto1)
    ingreso1.addProducto(producto2)
    admin1.addTransaccionRevisa(ingreso1)
    root['transacciones'].append(ingreso1)

    # Crear egreso
    egreso1 = Egreso("T002", 300.0, fecha2, "Pago a proveedor Y")
    trabajador1.addTransaccion(egreso1)
    admin1.addTransaccionRevisa(egreso1)
    root['transacciones'].append(egreso1)

# Función para listar clientes
def listarClientes(root):
    print("Listado de Clientes:")
    for c in root['clientes']:
        print(c)

# Función para listar transacciones entre dos fechas
def listarTransaccionesPorFecha(root, inicioStr, finStr):
    inicio = datetime.strptime(inicioStr, "%Y-%m-%d")
    fin = datetime.strptime(finStr, "%Y-%m-%d")
    print(f"\nTransacciones entre {inicioStr} y {finStr}:")
    for t in root['transacciones']:
        if inicio <= t.fechaTransaccion <= fin:
            print(t)



In [5]:
# Función principal
def main():
    # Configurar almacenamiento ZODB
    storage = ZODB.FileStorage.FileStorage('base_datos.fs')
    db = ZODB.DB(storage)
    connection = db.open()
    root = connection.root()

    # Inicializar las listas persistentes si no existen
    if 'clientes' not in root:
        root['clientes'] = PersistentList()
    if 'trabajadores' not in root:
        root['trabajadores'] = PersistentList()
    if 'administradores' not in root:
        root['administradores'] = PersistentList()
    if 'transacciones' not in root:
        root['transacciones'] = PersistentList()
    if 'productos' not in root:
        root['productos'] = PersistentList()

    # Insertar datos de ejemplo
    insertarDatos(root)
    transaction.commit()

    # Realizar consultas
    listarClientes(root)
    listarTransaccionesPorFecha(root, "2025-02-01", "2025-02-28")

    # Cerrar conexión
    connection.close()
    db.close()

if __name__ == '__main__':
    main()


Listado de Clientes:
Cliente [Persona [cedula=001, nombre=Juan, apellido=Perez], direccion=Av. Siempre Viva 123, correoElectronico=juan@mail.com, telefono=123456789]
Cliente [Persona [cedula=002, nombre=Maria, apellido=Lopez], direccion=Calle Falsa 456, correoElectronico=maria@mail.com, telefono=987654321]

Transacciones entre 2025-02-01 y 2025-02-28:
Ingreso [Transaccion [idTrans=T001, monto=500.0, fechaTransaccion=2025-02-15 00:00:00], motivoIngreso=Venta de producto X]
Egreso [Transaccion [idTrans=T002, monto=300.0, fechaTransaccion=2025-02-20 00:00:00], motivoEgreso=Pago a proveedor Y]
